# Building a Kalman Filter

In this notebook, we'll create a Kalman Filter using filterpy library.
We will also code our own Kalman Filter.

To make it easier, we'll use a library called Filterpy.
The filterpy library can implement a full filter, but we'll also code it ourselves for better understanding.

If you find some sentences similar to this [Kalman Filter Book](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python), it's because I took some matrices and concepts from this amazing book that I highly recommend!

Anyway, let's worry about tracking Speedy, our self-driving car, in 1, and 2D, using Kalman Filters!

**In this notebook, you'll learn how to predict Speedy's future positions and create a real time Machine!**

# Part 1 - First Order Kalman Filter

The first part of our exercise will be to implement what's called a First Order Kalman Filter. Let's not confuse you with details right now, we'll go back to orders later.

## Step 1 - Let's call the filterpy library and track Speedy in 1D

We will start by tracking Speedy in X direction.<p>
That's it!<p>
We need to carefully design our matrices to know what we're talking about.

In [3]:
from filterpy.kalman import KalmanFilter
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise
import numpy as np

We will track position and velocity, so the state variable needs both of these. <p>The matrix formulation is:

$$ \mathbf x = \begin{bmatrix}x\\\dot x\end{bmatrix}$$

So now we have to design our state transition. The Newtonian equations for a time step are:

$$\begin{aligned} x_t &= x_{t-1} + v\Delta t \\
 v_t &= v_{t-1}\end{aligned}$$

Recall that we need to convert this into the linear equation

$$\begin{bmatrix}x\\\dot x\end{bmatrix} = \mathbf F\begin{bmatrix}x\\\dot x\end{bmatrix}$$

Setting

$$\mathbf F = \begin{bmatrix}1 &\Delta t\\ 0 & 1\end{bmatrix}$$

gives us the equations above and implements what's called the **Constant Velocity Model**.

Finally, we design the measurement function. The measurement function needs to implement

$$\mathbf z = \mathbf{Hx}$$

Our sensor still only reads position, so it should take the position from the state, and 0 out the velocity and acceleration, like so:

$$\mathbf H = \begin{bmatrix}1 & 0 \end{bmatrix}$$

In [ ]:
def OneDimensionKF(R, Q_std, dt):
    """ Create first order 2D Kalman filter."""
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.zeros(2) # initial state (position and velocity)
    kf.P = np.eye(2) * 1000  # initial uncertainty
    kf.R = np.eye(1) * R  # measurement noise covariance
    kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=Q_std**2)  # process noise covariance
    kf.F = np.array([[1, dt],
                     [0, 1]])  # state transition matrix
    kf.H = np.array([[1, 0]])  # measurement function
    return kf

In [27]:
measurements = [1, 2, 3, 4, 5]
Q_std = 1
R_std =5
dt = 1.

kf = OneDimensionKF(R_std, Q_std, dt)

#TODO: Call Predict & Update in a loop
for idx,z in enumerate(measurements):
    kf.predict()
    print("predictiong ..............")
    print(f"predition for position {idx+1} is :",kf.x)
    print("updating ..............")
    kf.update(z)
    print(f"predition for position {idx+1} is :",kf.x)
    #print(f"prediction for position {idx+1} is :", kf.x)
    # print(kf.P)
    # print(kf.K)
    # print(kf.y)
    # print(kf.S)

predictiong ..............
predition for position 1 is : [0. 0.]
updating ..............
predition for position 1 is : [0.99875164 0.49946945]
predictiong ..............
predition for position 2 is : [1.49822108 0.49946945]
updating ..............
predition for position 2 is : [1.99753316 0.995209  ]
predictiong ..............
predition for position 3 is : [2.99274217 0.995209  ]
updating ..............
predition for position 3 is : [2.99879843 0.99894258]
predictiong ..............
predition for position 4 is : [3.99774101 0.99894258]
updating ..............
predition for position 4 is : [3.99936081 0.99973215]
predictiong ..............
predition for position 5 is : [4.99909296 0.99973215]
updating ..............
predition for position 5 is : [4.99968232 0.99999857]


## Step 2 - Now, let's track Speedy in 2D (x and y)

The equations are similar, and we're still implementing the Constant Acceleration Model.
The only thing we're doing is adding a y dimension.

We will track position and velocity, so the state variable needs both of these. <p>The matrix formulation is:

$$ \mathbf x = \begin{bmatrix}x\\\dot x\\y\\\dot y\\\end{bmatrix}$$

So now we have to design our state transition. The Newtonian equations for a time step are:

$$\begin{aligned} x_t &= x_{t-1} + v\Delta t \\
 v_xt &= v_x{t-1}\end{aligned}$$
$$\begin{aligned} y_t &= y_{t-1} + v\Delta t \\
 v_yt &= v_y{t-1}\end{aligned}$$

Recall that we need to convert this into the linear equation

$$\begin{bmatrix}x\\\dot x\\y\\\dot y\\\end{bmatrix} = \mathbf F\begin{bmatrix}x\\\dot x\\y\\\dot y\\\end{bmatrix}$$

Setting

$$\mathbf F = \begin{bmatrix}1& \Delta t& 0& 0\\0& 1& 0& 0\\0& 0& 1& \Delta t\\ 0& 0& 0& 1\end{bmatrix}$$

gives us the equations above and also implements the **Constant Velocity Model**, but considering v_x and v_y being constant.

Finally, we design the measurement function. The measurement function needs to implement

$$\mathbf z = \mathbf{Hx}$$

Our sensor still only reads position, so it should take the position from the state, and 0 out the velocity and acceleration, like so:

$$\mathbf H =
\begin{bmatrix} 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$

In [32]:
def TwoDimensionsKF(R_std, Q_std, dt):
    """ Create first order Kalman filter.
    Specify R and Q as floats."""
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.x = np.zeros(4)  # initial state (position and velocity)
    kf.P = np.eye(4) * 1000  # initial uncertainty
    kf.R = np.eye(2) * R_std**2  # measurement noise covariance
    q = Q_discrete_white_noise(dim = 2, dt = dt, var = Q_std**2)
    kf.Q = block_diag(q, q)
    kf.F = np.array([[1, dt, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, dt],
                     [0, 0, 0, 1]])  # state transition matrix
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 0, 1, 0]])  # measurement function
    return kf

measurements = [(1,1), (2,2), (3,3), (4,4), (5,5)]
Q_std =0.01
R_std = 5
dt = 1.

kf = TwoDimensionsKF(R_std, Q_std, dt)

for z in measurements:
    kf.predict()
    print("Predict...")
    print(kf.x)
    #print(kf.P)
    print("Measured: ",z)
    print("Update...")
    kf.update(z)
    print(kf.x)
    #print(kf.P)

Predict...
[0. 0. 0. 0.]
Measured:  (1, 1)
Update...
[0.98765432 0.49382718 0.98765432 0.49382718]
Predict...
[1.4814815  0.49382718 1.4814815  0.49382718]
Measured:  (2, 2)
Update...
[1.97767146 0.95693783 1.97767146 0.95693783]
Predict...
[2.93460929 0.95693783 2.93460929 0.95693783]
Measured:  (3, 3)
Update...
[2.98803619 0.98823076 2.98803619 0.98823076]
Predict...
[3.97626695 0.98823076 3.97626695 0.98823076]
Measured:  (4, 4)
Update...
[3.9926861  0.99517406 3.9926861  0.99517406]
Predict...
[4.98786015 0.99517406 4.98786015 0.99517406]
Measured:  (5, 5)
Update...
[4.99508506 0.99756077 4.99508506 0.99756077]


## Step 3 - Build Your Own 2D Kalman Filter
We are going to build a 2D filter that is going to track Speedy's x and y coordinates.
Why?
I just want to make sure that you understood how the maths work and how to build a Kalman Filter for any project! That's why you'll implement the full filter by yourself!

The filter will implement the maths of Kalman Filters to track something in 2D. 2D means we track x and y.
<p>We must define:<p>


*   x - Our state - It's a 4x1 matrix
*   x - Our state - It's a 4x1 matrix
*   P - Our covariance - It's a 4x4 matrix as well
*   Q - Our noise - It's a 4x4 matrix
*   F - Our transition matrix - It's a 4x4 matrix also
*   H - Our measurement transition matrix - It's a 2x4 matrix
*   R - Our measurement noise - It's a 4x4 matrix

Every time we measure something, we'll use a vector call Z.
*   Z - Our 2D Measurement - It's a 2x1 matrix
In the middle, we'll also use matrices such as the Kalman Gain or the Inverse matrix.
*   K - The Kalman Gain (should we trust the measurement or not) - It's a 4x2 matrix
*   y - The error - It's a 2x1 matrix
*   S - The system error - It's a 2x2 matrix
*   I - The identity matrix - It's a 4x4 matrix
*   SI - The Inverse of S - 2x2 as well

In [8]:
from numpy import eye, zeros, isscalar, dot
import numpy as np
from filterpy.common import pretty_str, reshape_z
from copy import deepcopy

class CustomKalmanFilter():
    def __init__(self, dim_x, dim_z, dim_u=0):
        self.dim_x = dim_x
        self.dim_z = dim_z
        self.dim_u = dim_u

        self.x = zeros(dim_x)             # state vector
        self.P = eye(dim_x) * 1000        # initial uncertainty
        self.Q = zeros((dim_x, dim_x))    # process noise covariance
        self.F = zeros((dim_x, dim_x))    # state transition matrix
        self.H = zeros((dim_z, dim_x))    # measurement function
        self.R = zeros((dim_z, dim_z))    # measurement noise covariance
        self.M = zeros((dim_x, dim_u))    # control input matrix
        self.z = zeros(dim_z)             # measurement vector

        # gain and residual are computed during the innovation step
        self.K = zeros((dim_x, dim_z))    # Kalman gain
        self.y = zeros(dim_z)             # residual (innovation)
        self.S = zeros((dim_z, dim_z))    # residual covariance
        self.SI = zeros((dim_z, dim_z))   # inverse of residual covariance

        # identity matrix. Do not alter this.
        self._I = eye(dim_x)

        # these will always be a copy of x, P after predict() is called
        self.x_prior = self.x.copy()
        self.P_prior = self.P.copy()

        # these will always be a copy of x, P after update() is called
        self.x_post = self.x.copy()
        self.P_post = self.P.copy()

    def predict(self, u=None, B=None, F=None, Q=None):
        """Predict the next state and covariance."""
        if F is not None:
            self.F = F
        if Q is not None:
            self.Q = Q

        if u is not None and B is not None:
            self.x = dot(self.F, self.x) + dot(B, u)
        else:
            self.x = dot(self.F, self.x)

        self.P = dot(dot(self.F, self.P), self.F.T) + self.Q

        # Save prior state
        self.x_prior = self.x.copy()
        self.P_prior = self.P.copy()

    def update(self, z, R=None, H=None):
        """Update the state with a new measurement."""
        if R is not None:
            self.R = R
        if H is not None:
            self.H = H

        self.y = z - dot(self.H, self.x)
        self.S = dot(dot(self.H, self.P), self.H.T) + self.R
        self.SI = np.linalg.inv(self.S)
        self.K = dot(dot(self.P, self.H.T), self.SI)

        self.x = self.x + dot(self.K, self.y)
        self.P = dot(self._I - dot(self.K, self.H), self.P)

        # Save posterior state
        self.x_post = self.x.copy()
        self.P_post = self.P.copy()
        

In [21]:
kf = CustomKalmanFilter(4,2)
kf.x = np.array([[0, 0, 0, 0]]).T
kf.P = np.eye(4) * 500.
dt = 1.0
kf.F = np.array([[1, dt, 0,  0],
                [0,  1, 0,  0],
                [0,  0, 1, dt],
                [0,  0, 0,  1]])

kf.H = np.array([[1, 0, 0, 0],
                    [0, 0, 1, 0]])

from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise

q = Q_discrete_white_noise(dim=2, dt=dt, var=0.001)
kf.Q = block_diag(q, q)
kf.R = np.array([[5., 0],
                      [0, 5]])

zs = [[1, 1], [2,2], [3,3], [4,4]]
for z in zs:
    kf.predict()
    print("Prediction...")
    print("Predicted X")
    print(kf.x_prior)
    print("Predicted P")
    print(kf.P_prior)
    kf.update(z)
    print("Measurement... z = ")
    print(z)
    print("Updated X")
    print(kf.x_post)
    print("Updated P")
    print(kf.P_post)

Prediction...
Predicted X
[[0.]
 [0.]
 [0.]
 [0.]]
Predicted P
[[1000.00025  500.0005     0.         0.     ]
 [ 500.0005   500.001      0.         0.     ]
 [   0.         0.      1000.00025  500.0005 ]
 [   0.         0.       500.0005   500.001  ]]
Measurement... z = 
[1, 1]
Updated X
[[0.99502488 0.99502488]
 [0.49751281 0.49751281]
 [0.99502488 0.99502488]
 [0.49751281 0.49751281]]
Updated P
[[  4.97512438   2.48756406   0.           0.        ]
 [  2.48756406 251.24434546   0.           0.        ]
 [  0.           0.           4.97512438   2.48756406]
 [  0.           0.           2.48756406 251.24434546]]
Prediction...
Predicted X
[[1.49253769 1.49253769]
 [0.49751281 0.49751281]
 [1.49253769 1.49253769]
 [0.49751281 0.49751281]]
Predicted P
[[261.19484796 253.73240952   0.           0.        ]
 [253.73240952 251.24534546   0.           0.        ]
 [  0.           0.         261.19484796 253.73240952]
 [  0.           0.         253.73240952 251.24534546]]
Measurement... z = 

# Part II - Implement the Constant Acceleration Model - Second Order Kalman Filter

So far, we have tracked Speedy's position and velocity. It was really great, but we assumed that it had a constant velocity. Sometimes, Speedy will accelerate and won't have a constant velocity; that's why we need to get one order higher and consider a constant acceleration.

No matter what we do, there is always an assumption to describe the movement.

What do I mean by order? <p>
An order is the number of derivatives required to model a system.

*   Consider a system that doesn't change; such as the height of The Eiffel Tower. There is no change, so there is no need for a derivative, and the order of the system is zero. We could express this in an equation as $x = 324$.

*   Now, let's consider a tracking problem. A first order system has a first derivative. For example, change of position is velocity, and we can write this as $$ v = \frac{dx}{dt}$$

which we integrate into the Newtonian equation

$$ x = vt + x_0.$$

This is what we called the **constant velocity** model, because of the assumption of a constant velocity.

*   Finally, a second order system has a second derivative. The second derivative of position is acceleration, with the equation

$$a = \frac{d^2x}{dt^2}$$

which we integrate into

$$ x = \frac{1}{2}at^2 +v_0t + x_0.$$

This is also known as the **constant acceleration** model.

## Step 1 - Second Order Kalman Filter in 1D

A second order Kalman filter tracks a second order system, such as position, velocity and acceleration. The state variable will be

$$ \mathbf x = \begin{bmatrix}x\\\dot x\\\ddot{x}\end{bmatrix}$$

So now we have to design our state transition. The Newtonian equations for a time step are:

$$\begin{aligned} x_t &= x_{t-1} + v_{t-1}\Delta t + 0.5a_{t-1} \Delta t^2 \\
 v_t &= v_{t-1} + a_{t-1}\Delta t \\
 a_t &= a_{t-1}\end{aligned}$$

We'll have to implement this in matrix multiplication or linear equation.

$$\begin{bmatrix}x\\\dot x\\\ddot{x}\end{bmatrix} = \mathbf F\begin{bmatrix}x\\\dot x\\\ddot{x}\end{bmatrix}$$

Setting

$$\mathbf F = \begin{bmatrix}1 & \Delta t &.5\Delta t^2\\
0 & 1 & \Delta t \\
0 & 0 & 1\end{bmatrix}$$

gives us the equations above.

Finally, we design the measurement function. The measurement function needs to implement

$$z = \mathbf{Hx}$$

Our sensor still only reads position, so it should take the position from the state, and 0 out the velocity, like so:

$$\mathbf H = \begin{bmatrix}1 & 0 & 0\end{bmatrix}$$

**This function constructs and returns a second order Kalman filter in 1D, only the position is tracked.**

In [11]:
def SecondOrderKF(R_std, Q_std, dt, P=100):
    """ Create second order Kalman filter. 
    Specify R and Q as floats."""
    
    kf = KalmanFilter(dim_x=3, dim_z=1)
    kf.x = np.zeros(3)
    kf.P = np.eye(3)*500
    kf.R = np.eye(1)*R_std**2
    kf.Q = Q_discrete_white_noise(3, dt, Q_std)
    kf.F = np.array([[1., dt, .5*dt*dt],
                     [0., 1.,       dt],
                     [0., 0.,       1.]])
    kf.H = np.array([[1., 0., 0.]])
    return kf

measurements = [1, 2, 3, 4, 6, 8, 10, 12]
Q_std = 0.01
R_std = 5
dt = 1.

kf = SecondOrderKF(R_std, Q_std, dt)
for z in measurements:
    kf.predict()
    print("Predict...")
    print(kf.x)
    #print(kf.P)
    print("Measured: ",z)
    print("Update...")
    kf.update(z)
    print(kf.x)

Predict...
[0. 0. 0.]
Measured:  1
Update...
[0.97826092 0.65217684 0.21739518]
Predict...
[1.73913535 0.86957202 0.21739518]
Measured:  2
Update...
[1.99376968 1.18380258 0.35825837]
Predict...
[3.35670144 1.54206095 0.35825837]
Measured:  3
Update...
[3.02399458 1.17913742 0.18021885]
Predict...
[4.29324142 1.35935627 0.18021885]
Measured:  4
Update...
[4.02955567 1.10753449 0.07391725]
Predict...
[5.17404879 1.18145174 0.07391725]
Measured:  6
Update...
[5.87877374 1.75019999 0.27213153]
Predict...
[7.76503949 2.02233152 0.27213153]
Measured:  8
Update...
[7.95307413 2.15121586 0.30979542]
Predict...
[10.2591877   2.46101128  0.30979542]
Measured:  10
Update...
[10.0654406  2.3464279  0.2810979]
Predict...
[12.55241745  2.6275258   0.2810979 ]
Measured:  12
Update...
[12.1667112   2.42746878  0.2372976 ]


## Challenge - Implement a 2D 2nd Order Kalman Filter

In [ ]:
def SecondOrderKF(R_std, Q_std, dt, P=100):
    """ Create second order Kalman filter.
    Specify R and Q as floats."""

    kf = KalmanFilter(dim_x= 6, dim_z=2)
    kf.x = np.zeros(6)  # initial state (position and velocity)
    kf.P = np.eye(6) * P
    kf.R = np.zeros((3,3))
    kf.Q = Q_discrete_white_noise()
    kf.F = [
        [1, 0, dt, 0, .5*dt*dt, 0],
        [0, 1, 0, dt, 0, .5*dt*dt],
        [0, 0, 1, 0, dt, 0],
        [0, 0, 0, 1, 0, dt],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
        ] # The only big problem will be here, you must write the equations by hand to make sure you didn't do any mistake
    kf.H = np.array([[1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0]]) 
    return kf

measurements = [(1,1), (2,2), (3,3), (4,4), (5,5)]
Q_std = 0.01
R_std = 5
dt = 1.

kf = SecondOrderKF(R_std, Q_std, dt)
for z in measurements:
    kf.predict()
    print("Predict...")
    print(kf.x)
    #print(kf.P)
    print("Measured: ",z)
    print("Update...")
    kf.update(z)
    print(kf.x)
    #print(kf.P)